In [41]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler

# Load the CSV data
df = pd.read_csv('../data/PEMfuelcell.csv')

# Display basic information about the dataset
print("Dataset shape:", df.shape)
print("\nFirst few rows of the dataset:")
print(df.head())
print("\nDataset columns:")
print(df.columns.tolist())
print("\nBasic statistics:")
print(df.describe())

Dataset shape: (1491, 9)

First few rows of the dataset:
     I           V    P             Q     T      Hydrogen        Oxygen  \
0  0.0  491.114619  0.0  8.580000e-35  20.0  8.410000e-38  0.000000e+00   
1  0.0  491.114619  0.0 -3.880000e-12  20.0  4.970000e-07  4.110000e-09   
2  0.0  491.114619  0.0 -7.750000e-12  20.0  1.140000e-06  1.130000e-08   
3  0.0  491.114619  0.0 -3.050000e-11  20.0  5.330000e-06  8.890000e-08   
4  0.0  491.114619  0.0 -6.400000e-11  20.0  1.160000e-05  2.240000e-07   

       RH anode  Rh Cathode  
0  0.000000e+00         0.5  
1  2.680000e-14         0.5  
2  8.050000e-14         0.5  
3  1.350000e-12         0.5  
4  5.360000e-12         0.5  

Dataset columns:
['I', 'V', 'P', 'Q', 'T', 'Hydrogen', 'Oxygen', 'RH anode', 'Rh Cathode']

Basic statistics:
                  I            V             P            Q            T  \
count  1.491000e+03  1491.000000  1.491000e+03  1491.000000  1491.000000   
mean   7.771881e+01   409.445420  2.473379e+01   

In [42]:
# Load the stack2_model.pkl model
import joblib

model_path = r'D:\Coding\Major-Project\new_\models\stack2_model.pkl'

try:
    # Try loading with joblib first
    stack2_model = joblib.load(model_path)
    print("Model loaded successfully with joblib!")
    print(f"Model type: {type(stack2_model)}")
except:
    try:
        # If joblib fails, try with pickle
        with open(model_path, 'rb') as f:
            stack2_model = pickle.load(f)
        print("Model loaded successfully with pickle!")
        print(f"Model type: {type(stack2_model)}")
    except Exception as e:
        print(f"Error loading model: {e}")
        stack2_model = None

# Display model information if loaded successfully
if stack2_model is not None:
    if hasattr(stack2_model, 'feature_names_in_'):
        print(f"Expected features: {stack2_model.feature_names_in_}")
    if hasattr(stack2_model, 'n_features_in_'):
        print(f"Number of features: {stack2_model.n_features_in_}")

Model loaded successfully with joblib!
Model type: <class 'sklearn.ensemble._stacking.StackingRegressor'>
Number of features: 8


In [43]:
# Inspect the saved model to see if it includes preprocessing or pipelines
print("MODEL INSPECTION")
print("="*40)

try:
    print(f"Model type: {type(stack2_model)}")
    # Show top-level attributes that often indicate pipelines/scalers
    attrs = ['steps', 'named_steps', 'preprocessor', 'transformers', 'estimators_', 'named_estimators_', 'final_estimator']
    for a in attrs:
        if hasattr(stack2_model, a):
            print(f"- Has attribute '{a}': {type(getattr(stack2_model, a))}")

    # If it's a stacking regressor, show named estimators
    if hasattr(stack2_model, 'named_estimators_'):
        print("\nNamed estimators:")
        for name, est in stack2_model.named_estimators_.items():
            print(f"  - {name}: {type(est)}")
            # If estimator is a pipeline, show its steps
            if hasattr(est, 'named_steps'):
                print(f"    Pipeline steps: {list(est.named_steps.keys())}")

    # Show keys in __dict__ for additional clues
    print('\nModel __dict__ keys:')
    pprint(sorted([k for k in stack2_model.__dict__.keys() if not k.startswith('_')])[:40])

except Exception as e:
    print(f"Error inspecting model: {e}")


MODEL INSPECTION
Model type: <class 'sklearn.ensemble._stacking.StackingRegressor'>
- Has attribute 'estimators_': <class 'list'>
- Has attribute 'named_estimators_': <class 'sklearn.utils._bunch.Bunch'>
- Has attribute 'final_estimator': <class 'sklearn.linear_model._ridge.RidgeCV'>

Named estimators:
  - enet: <class 'sklearn.pipeline.Pipeline'>
    Pipeline steps: ['standardscaler', 'elasticnet']
  - svr: <class 'sklearn.pipeline.Pipeline'>
    Pipeline steps: ['standardscaler', 'svr']
  - knn: <class 'sklearn.pipeline.Pipeline'>
    Pipeline steps: ['standardscaler', 'kneighborsregressor']
  - gbr: <class 'sklearn.ensemble._gb.GradientBoostingRegressor'>

Model __dict__ keys:
Error inspecting model: name 'pprint' is not defined


In [44]:
# Prepare input features as specified
# X = df[["I","P","Q","T","Hydrogen","Oxygen","RH anode","Rh Cathode"]].values
# y = df["V"].values

# Check if required columns exist in the dataset
required_features = ["I", "P", "Q", "T", "Hydrogen", "Oxygen", "RH anode", "Rh Cathode"]
print("Checking for required features in dataset:")
for feature in required_features:
    if feature in df.columns:
        print(f"✓ {feature} - Found")
    else:
        print(f"✗ {feature} - Not found")
        print(f"Available columns: {df.columns.tolist()}")

print(f"\nDataset columns: {df.columns.tolist()}")

# Extract features and target if they exist
if all(feature in df.columns for feature in required_features):
    X = df[required_features].values
    y = df["V"].values
    print(f"\nFeature matrix shape: {X.shape}")
    print(f"Target vector shape: {y.shape}")
    
    # Show sample of the data
    print(f"\nSample input values (first 5 rows):")
    feature_df = pd.DataFrame(X[:5], columns=required_features)
    print(feature_df)
    print(f"\nCorresponding target values: {y[:5]}")
else:
    print("Some required features are missing from the dataset!")
    X = None
    y = None

Checking for required features in dataset:
✓ I - Found
✓ P - Found
✓ Q - Found
✓ T - Found
✓ Hydrogen - Found
✓ Oxygen - Found
✓ RH anode - Found
✓ Rh Cathode - Found

Dataset columns: ['I', 'V', 'P', 'Q', 'T', 'Hydrogen', 'Oxygen', 'RH anode', 'Rh Cathode']

Feature matrix shape: (1491, 8)
Target vector shape: (1491,)

Sample input values (first 5 rows):
     I    P             Q     T      Hydrogen        Oxygen      RH anode  \
0  0.0  0.0  8.580000e-35  20.0  8.410000e-38  0.000000e+00  0.000000e+00   
1  0.0  0.0 -3.880000e-12  20.0  4.970000e-07  4.110000e-09  2.680000e-14   
2  0.0  0.0 -7.750000e-12  20.0  1.140000e-06  1.130000e-08  8.050000e-14   
3  0.0  0.0 -3.050000e-11  20.0  5.330000e-06  8.890000e-08  1.350000e-12   
4  0.0  0.0 -6.400000e-11  20.0  1.160000e-05  2.240000e-07  5.360000e-12   

   Rh Cathode  
0         0.5  
1         0.5  
2         0.5  
3         0.5  
4         0.5  

Corresponding target values: [491.1146188 491.1146188 491.1146188 491.1146189 491.

In [45]:
# Create sample input values based on dataset statistics for prediction
if stack2_model is not None and X is not None:
    print("Creating sample input values for prediction...")
    
    # Get statistics from the dataset to create realistic sample values
    print("\nDataset statistics for input features:")
    feature_stats = df[required_features].describe()
    print(feature_stats)
    
    # Create sample input values using different percentiles from the dataset
    sample_inputs = []
    
    # Sample 1: Using mean values
    mean_values = df[required_features].mean().values
    sample_inputs.append(mean_values)
    
    # Sample 2: Using 25th percentile values
    q25_values = df[required_features].quantile(0.25).values
    sample_inputs.append(q25_values)
    
    # Sample 3: Using 75th percentile values
    q75_values = df[required_features].quantile(0.75).values
    sample_inputs.append(q75_values)
    
    # Sample 4: Using median values
    median_values = df[required_features].median().values
    sample_inputs.append(median_values)
    
    # Sample 5: Custom realistic values (mix of different percentiles)
    custom_values = [
        np.mean([mean_values[i], q75_values[i]]) for i in range(len(mean_values))
    ]
    sample_inputs.append(custom_values)
    
    # Convert to numpy array
    sample_inputs = np.array(sample_inputs)
    
    print(f"\nSample inputs shape: {sample_inputs.shape}")
    print("\nSample input values:")
    sample_df = pd.DataFrame(sample_inputs, 
                           columns=required_features,
                           index=['Mean', '25th Percentile', '75th Percentile', 'Median', 'Custom Mix'])
    print(sample_df)
    
else:
    print("Cannot create sample inputs - model or data not available")

Creating sample input values for prediction...

Dataset statistics for input features:
                  I             P            Q            T      Hydrogen  \
count  1.491000e+03  1.491000e+03  1491.000000  1491.000000  1.491000e+03   
mean   7.771881e+01  2.473379e+01    14.318921    49.767306  3.825091e-03   
std    9.512274e+01  2.407938e+01    25.839677    23.605715  2.123214e-03   
min   -1.010000e-28 -5.050000e-29    -0.000006    20.000000  8.410000e-38   
25%    1.624014e+00  7.523455e-01     0.060226    21.596358  2.162324e-03   
50%    4.199939e+01  1.775851e+01     3.317269    58.815599  3.099518e-03   
75%    1.024381e+02  4.090923e+01    10.618404    66.948830  4.700892e-03   
max    3.559612e+02  7.272888e+01   106.208556    84.191731  9.400787e-03   

            Oxygen     RH anode   Rh Cathode  
count  1491.000000  1491.000000  1491.000000  
mean      0.027936     0.952695     0.975556  
std       0.025840     0.213035     0.106047  
min       0.000000     0.000000

In [46]:
# Fit a scaler on the training features and compare predictions with/without scaling
from sklearn.preprocessing import StandardScaler

print("Fitting StandardScaler on training features (X) and comparing predictions...")
scaler = StandardScaler()
scaler.fit(X)

# Predictions without scaling (current approach)
pred_no_scale = stack2_model.predict(sample_inputs)

# Predictions with scaling
sample_inputs_scaled = scaler.transform(sample_inputs)
pred_with_scale = stack2_model.predict(sample_inputs_scaled)

print('\nPredictions without scaling:')
print(pred_no_scale)
print('\nPredictions with scaling:')
print(pred_with_scale)

# Show difference per sample
print('\nDifference (with_scale - no_scale):')
print(pred_with_scale - pred_no_scale)

# Replace global predictions variable to scaled predictions for subsequent analysis
predictions = pred_with_scale
print('\nUsing scaled predictions for further analysis in the notebook (predictions variable updated).')


Fitting StandardScaler on training features (X) and comparing predictions...

Predictions without scaling:
[395.46027551 460.82881483 398.49552107 422.78820028 395.47757508]

Predictions with scaling:
[492.10699246 505.20648425 376.58862864 498.25010417 380.06453816]

Difference (with_scale - no_scale):
[ 96.64671695  44.37766941 -21.90689243  75.46190388 -15.41303692]

Using scaled predictions for further analysis in the notebook (predictions variable updated).


In [47]:
# Make predictions using the loaded model
if stack2_model is not None and 'sample_inputs' in locals():
    try:
        print("Making predictions with the stack2_model...")
        
        # Make predictions
        predictions = stack2_model.predict(sample_inputs)
        
        print(f"\nPredicted voltage values:")
        print(f"Predictions shape: {predictions.shape}")
        
        # Create a results DataFrame
        results_df = pd.DataFrame({
            'Input_Type': ['Mean', '25th Percentile', '75th Percentile', 'Median', 'Custom Mix'],
            'Predicted_Voltage': predictions
        })
        
        # Add input features to results for reference
        for i, feature in enumerate(required_features):
            results_df[f'Input_{feature}'] = sample_inputs[:, i]
        
        print("\nPrediction Results:")
        print(results_df)
        
        # Show comparison with actual voltage range from dataset
        actual_voltage_stats = df["V"].describe()
        print(f"\nActual voltage statistics from dataset:")
        print(actual_voltage_stats)
        
        print(f"\nPredicted voltage range: {predictions.min():.4f} to {predictions.max():.4f}")
        print(f"Actual voltage range: {actual_voltage_stats['min']:.4f} to {actual_voltage_stats['max']:.4f}")
        
    except Exception as e:
        print(f"Error making predictions: {e}")
        print("This might be due to:")
        print("- Model expecting different input format")
        print("- Feature scaling requirements")
        print("- Model compatibility issues")
        
else:
    print("Cannot make predictions - model or sample inputs not available")

Making predictions with the stack2_model...

Predicted voltage values:
Predictions shape: (5,)

Prediction Results:
        Input_Type  Predicted_Voltage     Input_I    Input_P    Input_Q  \
0             Mean         395.460276   77.718810  24.733794  14.318921   
1  25th Percentile         460.828815    1.624014   0.752345   0.060226   
2  75th Percentile         398.495521  102.438099  40.909228  10.618404   
3           Median         422.788200   41.999390  17.758512   3.317269   
4       Custom Mix         395.477575   90.078455  32.821511  12.468663   

     Input_T  Input_Hydrogen  Input_Oxygen  Input_RH anode  Input_Rh Cathode  
0  49.767306        0.003825      0.027936        0.952695          0.975556  
1  21.596358        0.002162      0.013887        1.000312          0.999995  
2  66.948830        0.004701      0.035989        1.004916          1.000041  
3  58.815599        0.003100      0.015105        1.000922          1.000011  
4  58.358068        0.004263      0.03

In [48]:
# Create a convenient prediction function for future use (updated to support optional scaler)
from typing import Optional

def predict_voltage(current_values, model=stack2_model, scaler: Optional[object]=None):
    """
    Predict voltage using the stack2_model. Optionally apply a scaler.

    Parameters:
    - current_values: list or array of 8 input values in order:
                   [I, P, Q, T, Hydrogen, Oxygen, RH anode, Rh Cathode]
    - model: trained model (StackingRegressor)
    - scaler: an object with `transform()` like sklearn's StandardScaler (optional)

    Returns:
    - predicted_voltage: float or error message
    """
    if model is None:
        return "Model not loaded"

    # Ensure input is in correct format
    try:
        arr = np.array(current_values, dtype=float)
    except Exception as e:
        return f"Error converting inputs to array: {e}"

    if arr.ndim == 1:
        if arr.shape[0] != 8:
            return f"Error: Expected 8 input values, got {arr.shape[0]}"
        input_array = arr.reshape(1, -1)
    else:
        # allow passing already 2D arrays
        input_array = arr
        if input_array.shape[1] != 8:
            return f"Error: Expected 8 features, got {input_array.shape[1]}"

    # Apply scaler if provided
    if scaler is not None:
        try:
            input_array = scaler.transform(input_array)
        except Exception as e:
            return f"Scaling error: {e}"

    try:
        prediction = model.predict(input_array)
        return float(prediction[0])
    except Exception as e:
        return f"Prediction error: {e}"

# Small note: use predict_voltage(..., scaler=scaler) to apply training scaling when predicting.
print('Updated predict_voltage() now supports an optional scaler parameter.')


Updated predict_voltage() now supports an optional scaler parameter.


In [49]:
# How to Get Better Voltage Values

Based on the fuel cell physics and your current predictions, here are strategies to improve voltage output:

SyntaxError: invalid syntax (1845054439.py, line 3)

In [ ]:
# Analyze current predictions to understand voltage patterns
print("ANALYSIS: Understanding Voltage Patterns")
print("="*50)

# Find the highest voltage predictions from our dataset
high_voltage_indices = np.argsort(y)[-10:]  # Top 10 highest voltages
low_voltage_indices = np.argsort(y)[:10]    # Bottom 10 lowest voltages

print("TOP 10 HIGHEST VOLTAGE CONDITIONS:")
high_voltage_conditions = df.iloc[high_voltage_indices][required_features + ['V']]
print(high_voltage_conditions)

print("\nTOP 10 LOWEST VOLTAGE CONDITIONS:")
low_voltage_conditions = df.iloc[low_voltage_indices][required_features + ['V']]
print(low_voltage_conditions)

# Calculate correlations to understand which factors most affect voltage
print("\nCORRELATION ANALYSIS:")
correlations = df[required_features + ['V']].corr()['V'].sort_values(ascending=False)
print("Feature correlations with Voltage (V):")
for feature, corr in correlations.items():
    if feature != 'V':
        print(f"{feature:15}: {corr:+.4f}")
        
print(f"\nCurrent prediction range: {predictions.min():.2f} - {predictions.max():.2f} V")
print(f"Actual dataset range:     {y.min():.2f} - {y.max():.2f} V")

ANALYSIS: Understanding Voltage Patterns
TOP 10 HIGHEST VOLTAGE CONDITIONS:
                 I              P         Q          T  Hydrogen   Oxygen  \
838 -6.230000e-207 -3.130000e-207  0.000006  59.016533  0.002014  0.01389   
839  -2.670000e-51  -1.340000e-51  0.000005  58.917949  0.002013  0.01389   
840   2.670000e-51   1.340000e-51  0.000005  58.730359  0.002012  0.01389   
847   1.836825e-02   9.218615e-03  0.000008  58.648037  0.002012  0.01389   
846   1.204614e-02   6.045800e-03  0.000007  58.648254  0.002012  0.01389   
845   5.724265e-03   2.872985e-03  0.000006  58.648471  0.002012  0.01389   
841  -3.710000e-68  -1.860000e-68  0.000005  58.654474  0.002012  0.01389   
842   1.180000e-38   5.900000e-39  0.000005  58.651394  0.002012  0.01389   
843  -5.880000e-39  -2.950000e-39  0.000005  58.649543  0.002012  0.01389   
844   3.670000e-40   1.840000e-40  0.000005  58.648688  0.002012  0.01389   

     RH anode  Rh Cathode           V  
838  1.000938    1.000000  501.69838

In [ ]:
# Strategy 1: Optimize operating conditions for higher voltage
print("\nSTRATEGY 1: OPTIMAL CONDITIONS FOR HIGH VOLTAGE")
print("="*55)

# Find the average conditions that produce highest voltages (top 20%)
top_20_percent_threshold = np.percentile(y, 80)
high_voltage_mask = y >= top_20_percent_threshold

optimal_conditions = df[high_voltage_mask][required_features].mean()
print("OPTIMAL AVERAGE CONDITIONS (from top 20% voltage samples):")
for feature, value in optimal_conditions.items():
    print(f"{feature:15}: {value:.6f}")

# Test prediction with optimal conditions
optimal_voltage = predict_voltage(optimal_conditions.values)
print(f"\nPredicted voltage with optimal conditions: {optimal_voltage:.4f} V")

# Strategy 2: Fine-tune individual parameters
print("\nSTRATEGY 2: PARAMETER OPTIMIZATION")
print("="*40)

# Start with optimal conditions and adjust key parameters
base_conditions = optimal_conditions.values.copy()
improvements = {}

# Test different parameter ranges
parameter_tests = {
    'Current (I)': (0, [5, 10, 15, 20, 25, 30]),  # Lower current often = higher voltage
    'Temperature (T)': (3, [20, 25, 30, 35, 40, 45]),  # Lower temp often = higher voltage  
    'Hydrogen': (4, [0.008, 0.009, 0.0095]),  # Higher hydrogen = better
    'RH anode': (6, [0.99, 1.0, 1.01, 1.02]),  # Optimal humidity
    'Rh Cathode': (7, [0.99, 1.0, 1.01, 1.02])  # Optimal humidity
}

for param_name, (index, test_values) in parameter_tests.items():
    best_voltage = optimal_voltage
    best_value = base_conditions[index]
    
    for test_val in test_values:
        test_conditions = base_conditions.copy()
        test_conditions[index] = test_val
        test_voltage = predict_voltage(test_conditions)
        
        if isinstance(test_voltage, float) and test_voltage > best_voltage:
            best_voltage = test_voltage
            best_value = test_val
    
    improvements[param_name] = (best_value, best_voltage)
    print(f"{param_name:15}: {best_value:.6f} → {best_voltage:.4f} V")

# Find the single best parameter adjustment
best_improvement = max(improvements.values(), key=lambda x: x[1])
print(f"\nBest single improvement: {best_improvement[1]:.4f} V")


STRATEGY 1: OPTIMAL CONDITIONS FOR HIGH VOLTAGE
OPTIMAL AVERAGE CONDITIONS (from top 20% voltage samples):
I              : 0.001350
P              : 0.000668
Q              : 0.000090
T              : 25.129679
Hydrogen       : 0.002543
Oxygen         : 0.008070
RH anode       : 0.910522
Rh Cathode     : 0.954957

Predicted voltage with optimal conditions: 491.3234 V

STRATEGY 2: PARAMETER OPTIMIZATION
Current (I)    : 0.001350 → 491.3234 V
Temperature (T): 25.129679 → 491.3234 V
Hydrogen       : 0.002543 → 491.3234 V
RH anode       : 1.010000 → 492.3117 V
Rh Cathode     : 1.000000 → 491.3868 V

Best single improvement: 492.3117 V


In [ ]:
# Explore sensitivity of predictions to the flow rate (Q)
# We will vary Q across a meaningful range while holding other features at the 'Median' sample

print('Testing sensitivity to flow rate (Q)')
median_sample = sample_inputs[3].copy()  # index 3 = Median

q_values = np.linspace(df['Q'].quantile(0.01), df['Q'].quantile(0.99), 15)
results = []
for q in q_values:
    test = median_sample.copy()
    test[2] = q  # Q is the third feature (index 2)
    # Use scaler when predicting
    v = predict_voltage(test, model=stack2_model, scaler=scaler)
    results.append((q, v))

# Show results in a small DataFrame
q_df = pd.DataFrame(results, columns=['Q', 'Predicted_V'])
print(q_df)

# Compute correlation between Q and predicted voltage (should not be ~0)
corr_q = q_df['Q'].corr(q_df['Predicted_V'])
print('\nCorrelation between Q and predicted voltage:', corr_q)

# If correlation is near zero, it suggests the model ignores Q; otherwise Q affects predictions.
if abs(corr_q) < 0.01:
    print('\nWARNING: Flow rate (Q) appears to have little to no effect on predictions (correlation ~ 0).')
    print('Consider retraining the model with feature engineering or verifying training data scaling.')
else:
    print('\nOK: Flow rate (Q) affects predicted voltage (non-zero correlation).')


Testing sensitivity to flow rate (Q)
             Q  Predicted_V
0    -0.000005   498.250316
1     7.586314   498.249832
2    15.172633   476.497023
3    22.758953   467.173214
4    30.345272   461.890379
5    37.931591   457.969960
6    45.517910   453.197893
7    53.104230   451.963720
8    60.690549   449.314284
9    68.276868   446.461586
10   75.863187   444.623494
11   83.449507   441.418649
12   91.035826   441.397720
13   98.622145   439.280200
14  106.208464   437.036720

Correlation between Q and predicted voltage: -0.9210107665197783

OK: Flow rate (Q) affects predicted voltage (non-zero correlation).


In [ ]:
# Strategy 3: Combined optimization approach
print("\nSTRATEGY 3: COMBINED OPTIMIZATION")
print("="*40)

# Apply multiple optimizations together
optimized_conditions = optimal_conditions.values.copy()

# Apply the best improvements found
for param_name, (index, _) in parameter_tests.items():
    if param_name in improvements:
        optimized_conditions[index] = improvements[param_name][0]

final_optimized_voltage = predict_voltage(optimized_conditions)

print("FINAL OPTIMIZED CONDITIONS:")
for i, (feature, value) in enumerate(zip(required_features, optimized_conditions)):
    print(f"{feature:15}: {value:.6f}")

print(f"\nVoltage Progression:")
print(f"Original optimal:      {optimal_voltage:.4f} V")
print(f"Final optimized:       {final_optimized_voltage:.4f} V")
print(f"Improvement:          +{final_optimized_voltage - optimal_voltage:.4f} V")

# Strategy 4: Create multiple high-voltage scenarios
print("\nSTRATEGY 4: MULTIPLE HIGH-VOLTAGE SCENARIOS")
print("="*50)

scenarios = {
    'Low Current, Optimal Conditions': [
        5.0,      # Very low current
        2.0,      # Low power
        1.0,      # Low flow rate
        25.0,     # Low temperature
        0.009,    # High hydrogen
        0.02,     # Moderate oxygen
        1.01,     # Optimal humidity anode
        1.01      # Optimal humidity cathode
    ],
    'Minimal Load, High Efficiency': [
        1.0,      # Minimal current
        0.5,      # Minimal power
        0.5,      # Minimal flow
        22.0,     # Cool temperature
        0.0095,   # Very high hydrogen
        0.015,    # Optimal oxygen
        1.02,     # High humidity anode
        1.015     # High humidity cathode
    ],
    'Balanced High Performance': [
        15.0,     # Moderate current
        8.0,      # Moderate power
        3.0,      # Moderate flow
        30.0,     # Moderate temperature
        0.008,    # High hydrogen
        0.025,    # Good oxygen
        1.0,      # Perfect humidity anode
        1.0       # Perfect humidity cathode
    ]
}

print("HIGH-VOLTAGE SCENARIOS:")
for scenario_name, conditions in scenarios.items():
    voltage = predict_voltage(conditions)
    print(f"\n{scenario_name}:")
    print(f"  Predicted voltage: {voltage:.4f} V")
    print(f"  Conditions: {conditions}")


STRATEGY 3: COMBINED OPTIMIZATION
FINAL OPTIMIZED CONDITIONS:
I              : 0.001350
P              : 0.000668
Q              : 0.000090
T              : 25.129679
Hydrogen       : 0.002543
Oxygen         : 0.008070
RH anode       : 1.010000
Rh Cathode     : 1.000000

Voltage Progression:
Original optimal:      491.3234 V
Final optimized:       492.3638 V
Improvement:          +1.0404 V

STRATEGY 4: MULTIPLE HIGH-VOLTAGE SCENARIOS
HIGH-VOLTAGE SCENARIOS:

Low Current, Optimal Conditions:
  Predicted voltage: 435.9381 V
  Conditions: [5.0, 2.0, 1.0, 25.0, 0.009, 0.02, 1.01, 1.01]

Minimal Load, High Efficiency:
  Predicted voltage: 444.7428 V
  Conditions: [1.0, 0.5, 0.5, 22.0, 0.0095, 0.015, 1.02, 1.015]

Balanced High Performance:
  Predicted voltage: 418.6206 V
  Conditions: [15.0, 8.0, 3.0, 30.0, 0.008, 0.025, 1.0, 1.0]


In [ ]:
# Summary and Recommendations
print("\n" + "="*60)
print("SUMMARY: HOW TO GET BETTER VOLTAGE VALUES")
print("="*60)

print("\n🔋 KEY STRATEGIES FOR HIGHER VOLTAGE:")
print("\n1. REDUCE CURRENT DRAW")
print("   - Lower current (I) typically increases voltage")
print("   - Aim for I < 20A for maximum voltage")

print("\n2. OPTIMIZE TEMPERATURE")
print("   - Keep temperature low (20-30°C)")
print("   - Higher temperatures reduce voltage")

print("\n3. MAXIMIZE FUEL SUPPLY")
print("   - Increase Hydrogen concentration (>0.008)")
print("   - Maintain optimal Oxygen levels (0.015-0.025)")

print("\n4. CONTROL HUMIDITY")
print("   - Keep RH anode and Rh Cathode close to 1.0 (100%)")
print("   - Slight over-humidification (1.01-1.02) can help")

print("\n5. MINIMIZE FLOW RATES")
print("   - Lower Q (flow rate) often correlates with higher voltage")
print("   - Balance between efficiency and cooling needs")

print("\n📊 VOLTAGE IMPROVEMENT POTENTIAL:")
print(f"   Current range:     {predictions.min():.1f} - {predictions.max():.1f} V")
print(f"   Dataset maximum:   {y.max():.1f} V")
print(f"   Potential gain:    +{y.max() - predictions.max():.1f} V")

print("\n⚡ PRACTICAL RECOMMENDATIONS:")
print("   - Start with the 'Low Current, Optimal Conditions' scenario")
print("   - Gradually adjust parameters while monitoring voltage")
print("   - Use the predict_voltage() function to test new combinations")
print("   - Focus on the parameters with highest correlation to voltage")


SUMMARY: HOW TO GET BETTER VOLTAGE VALUES

🔋 KEY STRATEGIES FOR HIGHER VOLTAGE:

1. REDUCE CURRENT DRAW
   - Lower current (I) typically increases voltage
   - Aim for I < 20A for maximum voltage

2. OPTIMIZE TEMPERATURE
   - Keep temperature low (20-30°C)
   - Higher temperatures reduce voltage

3. MAXIMIZE FUEL SUPPLY
   - Increase Hydrogen concentration (>0.008)
   - Maintain optimal Oxygen levels (0.015-0.025)

4. CONTROL HUMIDITY
   - Keep RH anode and Rh Cathode close to 1.0 (100%)
   - Slight over-humidification (1.01-1.02) can help

5. MINIMIZE FLOW RATES
   - Lower Q (flow rate) often correlates with higher voltage
   - Balance between efficiency and cooling needs

📊 VOLTAGE IMPROVEMENT POTENTIAL:
   Current range:     395.5 - 460.8 V
   Dataset maximum:   501.9 V
   Potential gain:    +41.1 V

⚡ PRACTICAL RECOMMENDATIONS:
   - Start with the 'Low Current, Optimal Conditions' scenario
   - Gradually adjust parameters while monitoring voltage
   - Use the predict_voltage() fun

In [ ]:
# EVALUATION: Train/test split and model metrics (raw vs scaled vs pipeline)
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
import joblib

print('Running model evaluation on a held-out test split...')

# Prepare data (X, y already defined earlier)
X_all = X
y_all = y

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2, random_state=42)

# Fit scaler on training set
eval_scaler = StandardScaler()
eval_scaler.fit(X_train)

# 1) Evaluate raw model (no external scaling)
try:
    y_pred_raw = stack2_model.predict(X_test)
    raw_mae = mean_absolute_error(y_test, y_pred_raw)
    raw_rmse = float(np.sqrt(mean_squared_error(y_test, y_pred_raw)))
    raw_r2 = r2_score(y_test, y_pred_raw)
except Exception as e:
    y_pred_raw = None
    raw_mae = raw_rmse = raw_r2 = str(e)

# 2) Evaluate model with external scaling (apply eval_scaler to inputs)
try:
    X_test_scaled = eval_scaler.transform(X_test)
    y_pred_scaled = stack2_model.predict(X_test_scaled)
    scaled_mae = mean_absolute_error(y_test, y_pred_scaled)
    scaled_rmse = float(np.sqrt(mean_squared_error(y_test, y_pred_scaled)))
    scaled_r2 = r2_score(y_test, y_pred_scaled)
except Exception as e:
    y_pred_scaled = None
    scaled_mae = scaled_rmse = scaled_r2 = str(e)

# 3) Evaluate wrapped pipeline (scaler + model)
try:
    pipeline = Pipeline([('scaler', eval_scaler), ('stack', stack2_model)])
    y_pred_pipe = pipeline.predict(X_test)
    pipe_mae = mean_absolute_error(y_test, y_pred_pipe)
    pipe_rmse = float(np.sqrt(mean_squared_error(y_test, y_pred_pipe)))
    pipe_r2 = r2_score(y_test, y_pred_pipe)
except Exception as e:
    y_pred_pipe = None
    pipe_mae = pipe_rmse = pipe_r2 = str(e)

print('\nEvaluation Results (held-out test set):')
print('--------------------------------------')
print('Approach             MAE       RMSE      R2')
print('Raw model:      ', f"{raw_mae!s:9}", f"{raw_rmse!s:9}", f"{raw_r2!s:9}")
print('Scaled inputs:  ', f"{scaled_mae!s:9}", f"{scaled_rmse!s:9}", f"{scaled_r2!s:9}")
print('Scaler+Model:   ', f"{pipe_mae!s:9}", f"{pipe_rmse!s:9}", f"{pipe_r2!s:9}")

# Show a small sample comparison
print('\nSample predictions (first 10 test rows):')
comp_df = pd.DataFrame({
    'y_true': y_test[:10],
    'pred_raw': (y_pred_raw[:10] if y_pred_raw is not None else [None]*10),
    'pred_scaled': (y_pred_scaled[:10] if y_pred_scaled is not None else [None]*10),
    'pred_pipeline': (y_pred_pipe[:10] if y_pred_pipe is not None else [None]*10)
})
print(comp_df)

# Save pipeline to disk if it gives best performance (ask user before saving)
best = None
try:
    # choose best by RMSE (numeric)
    scores = {}
    for name, rmse in [('raw', raw_rmse), ('scaled', scaled_rmse), ('pipe', pipe_rmse)]:
        if isinstance(rmse, (int, float)):
            scores[name] = rmse
    if scores:
        best = min(scores, key=scores.get)
        print(f"\nBest approach by RMSE: {best} (RMSE={scores[best]:.4f})")
    else:
        print('\nCould not determine numeric RMSE values to pick best approach.')
except Exception:
    pass

# Keep evaluation outputs in notebook variables
evaluation_results = {
    'raw': {'mae': raw_mae, 'rmse': raw_rmse, 'r2': raw_r2},
    'scaled': {'mae': scaled_mae, 'rmse': scaled_rmse, 'r2': scaled_r2},
    'pipeline': {'mae': pipe_mae, 'rmse': pipe_rmse, 'r2': pipe_r2},
    'best': best
}

print('\nEvaluation complete. Use `evaluation_results` to inspect numeric values programmatically.')


Running model evaluation on a held-out test split...

Evaluation Results (held-out test set):
--------------------------------------
Approach             MAE       RMSE      R2
Raw model:       0.4519715173486524 0.78767173132978 0.9998869172314876
Scaled inputs:   49.510567590731 57.652223379595206 0.39418777213831924
Scaler+Model:    49.510567590731 57.652223379595206 0.39418777213831924

Sample predictions (first 10 test rows):
       y_true    pred_raw  pred_scaled  pred_pipeline
0  424.181776  423.729303   499.076790     499.076790
1  492.609584  492.526060   506.288470     506.288470
2  492.344650  492.434131   506.904031     506.904031
3  433.256625  433.643437   501.563886     501.563886
4  430.302098  429.835245   502.887823     502.887823
5  413.973812  412.948730   496.153435     496.153435
6  425.204246  422.397910   501.824927     501.824927
7  266.069958  266.669490   315.052535     315.052535
8  492.464394  492.321561   508.453636     508.453636
9  394.917489  394.721628

In [ ]:
# BEST SOLUTION: Create an explicit raw-input pipeline and save it
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
import joblib

save_path = r'D:\Coding\Major-Project\new_\models\stack2_model_raw_input.pkl'

# Use FunctionTransformer with default func=None (identity) which is picklable
identity = FunctionTransformer(validate=False)
raw_pipeline = Pipeline([('identity', identity), ('stack', stack2_model)])

# Test the pipeline quickly on a sample
sample = X[:3]
print('Sample predictions from raw_input pipeline (first 3 rows):')
print(raw_pipeline.predict(sample))

# Save to disk
joblib.dump(raw_pipeline, save_path)
print(f"Saved raw-input pipeline to: {save_path}")

print('\nUsage:')
print("- Load with joblib.load('<path>') and call .predict(raw_feature_array)")
print("- Do NOT apply an external scaler before calling .predict() because base estimators already handle scaling.")


Sample predictions from raw_input pipeline (first 3 rows):
[491.17060459 491.17057755 491.17054258]
Saved raw-input pipeline to: D:\Coding\Major-Project\new_\models\stack2_model_raw_input.pkl

Usage:
- Load with joblib.load('<path>') and call .predict(raw_feature_array)
- Do NOT apply an external scaler before calling .predict() because base estimators already handle scaling.


In [ ]:
# HYDROGEN SENSITIVITY TEST + PERMUTATION IMPORTANCE
print('HYDROGEN SENSITIVITY TEST')
print('='*60)

# Baselines: median sample and optimal conditions (if available)
baseline_median = sample_inputs[3].copy() if 'sample_inputs' in globals() else X.mean(axis=0)
baseline_optimal = optimized_conditions.copy() if 'optimized_conditions' in globals() else baseline_median

# Hydrogen range (including very low values)
h_values = np.concatenate((np.linspace(1e-6, 0.001, 6), np.linspace(0.001, 0.01, 10)))

results = []
for h in h_values:
    s = baseline_median.copy()
    s[4] = h  # Hydrogen is feature index 4
    pred = predict_voltage(s, model=stack2_model, scaler=None)
    results.append((h, float(pred)))

median_df = pd.DataFrame(results, columns=['Hydrogen', 'Pred_MedianBase'])

results_opt = []
for h in h_values:
    s = baseline_optimal.copy()
    s[4] = h
    pred = predict_voltage(s, model=stack2_model, scaler=None)
    results_opt.append((h, float(pred)))
opt_df = pd.DataFrame(results_opt, columns=['Hydrogen', 'Pred_OptimalBase'])

print('\nMedian baseline sensitivity:')
print(median_df)
print('\nOptimal baseline sensitivity:')
print(opt_df)

# Correlations
corr_med = median_df['Hydrogen'].corr(median_df['Pred_MedianBase'])
corr_opt = opt_df['Hydrogen'].corr(opt_df['Pred_OptimalBase'])
print(f"\nCorrelation (Hydrogen vs Predicted) on median baseline: {corr_med:.4f}")
print(f"Correlation (Hydrogen vs Predicted) on optimal baseline: {corr_opt:.4f}")

# Permutation importance on test set (if X_test exists from evaluation cell)
print('\nPERMUTATION IMPORTANCE ON TEST SET')
print('-'*40)
try:
    from sklearn.inspection import permutation_importance
    features = required_features
    # Use raw model predictions (stack2_model expects raw inputs)
    r = permutation_importance(stack2_model, X_test, y_test, n_repeats=10, random_state=42, scoring='neg_mean_squared_error')
    imp_df = pd.DataFrame({'feature': features, 'importance_mean': r.importances_mean, 'importance_std': r.importances_std})
    imp_df = imp_df.sort_values('importance_mean', ascending=False)
    print(imp_df)
    hydrogen_importance = imp_df.loc[imp_df['feature'] == 'Hydrogen', 'importance_mean'].values
    hydrogen_imp = float(hydrogen_importance[0]) if hydrogen_importance.size else None
    print(f"\nPermutation importance (mean) for 'Hydrogen': {hydrogen_imp}")
except Exception as e:
    print('Permutation importance failed:', e)
    imp_df = None

# Store results for inspection
sensitivity_results = {'median_df': median_df, 'opt_df': opt_df, 'perm_importances': imp_df}
print('\nHYDROGEN SENSITIVITY TEST COMPLETE')


HYDROGEN SENSITIVITY TEST

Median baseline sensitivity:
    Hydrogen  Pred_MedianBase
0   0.000001       418.121997
1   0.000201       417.579526
2   0.000401       417.584923
3   0.000600       417.207987
4   0.000800       416.900734
5   0.001000       416.657446
6   0.001000       416.657446
7   0.002000       416.169732
8   0.003000       422.712704
9   0.004000       418.436925
10  0.005000       417.565471
11  0.006000       409.060952
12  0.007000       408.654189
13  0.008000       407.049670
14  0.009000       405.019092
15  0.010000       403.908180

Optimal baseline sensitivity:
    Hydrogen  Pred_OptimalBase
0   0.000001        493.208269
1   0.000201        493.206390
2   0.000401        493.205731
3   0.000600        493.206175
4   0.000800        493.207599
5   0.001000        493.209876
6   0.001000        493.209876
7   0.002000        493.131152
8   0.003000        493.038774
9   0.004000        492.846147
10  0.005000        492.179282
11  0.006000        479.418366
